# Network Analysis Lab

Complete the following exercises to help solidify your understanding of network analysis.

In [ ]:
import networkx as nx
import nxviz
import community
import pandas as pd

## U.S. Mens Basketball Data Set

In the `us_mens_basketball.csv` data set, each row represents an single basketball player's participation in a single event at a single Olympics. 

In [ ]:
basketball = pd.read_csv('./data/us_mens_basketball.csv')

In [ ]:
basketball.head()

## 1. Transform this data set into one that can be turned into a graph where the entities are represented by the Name field and the relationships are represented by whether the players played in the same Olympics together (Games field).

Sort descending by the number of pairwise interactions. Which pair of players have competed in the most Olympics together?

## 2. Use the `from_pandas_edgelist` method to turn the data frame into a graph.

## 3. Compute and print the following graph statistics for the graph:

- Number of nodes
- Number of edges
- Average degree
- Density

## 4. Compute betweenness centrality for the graph and print the top 5 nodes with the highest centrality.

## 5. Compute Eigenvector centrality for the graph and print the top 5 nodes with the highest centrality.

## 6. Compute degree centrality for the graph and print the top 5 nodes with the highest centrality.

## 7. Generate a network visualization for the entire graph using a Kamada-Kawai force-directed layout.

## 8. Create and visualize an ego graph for the player with the highest betweenness centrality.

## 9. Identify the communities within the entire graph and produce another visualization of it with the nodes color-coded by the community they belong to.

## Bonus: Hierarchical Graphs

Thus far, we have analyzed graphs where the nodes represented individual players and the edges represented Olympic games that they have competed in together. We can analyze the data at a higher level if we wanted to, strippping out the players as entities and analyzing the data at the Games level. To do this, we would need to reconstruct the graph so that the *Games* field represents the entities and then use the player names as the edge criteria so that there would be an edge between two Olympic games if an player played in both of them. You already have the tools in your toolbox to be able to do this, so give it a try. 

### Create a graph with Games as the entities and then print out the graph statistics.

### Generate a network visualization of this graph using the layout of your choice.